In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster
import os


# Step 1: 去除時間
def remove_time(gpx_file: str) -> gpxpy.gpx.GPX:
    """從 GPX 檔案中移除所有軌跡點的時間戳記。"""
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                point.time = None
    print(f"    - 已移除時間戳記")
    return gpx


# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    """從 TXT 檔案讀取通訊點資料。"""
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        try:
            lat = float(str(row["緯度"]).replace("°", "").strip())
            lon = float(str(row["經度"]).replace("°", "").strip())
            ele = float(str(row["海拔（約）"]).replace("m", "").strip())
            label = f"{row['步道名稱']} {row['路標指示']}"
            points.append((label, lat, lon, ele))
        except (ValueError, KeyError) as e:
            print(f"      - ⚠️ 讀取通訊點時發生錯誤，請檢查 TXT 檔案格式：{e}")
            continue
    print(f"    - 成功讀取 {len(points)} 個通訊點")
    return points


# Step 2-2: 插入通訊點
def insert_points(
    gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    """將通訊點插入到 GPX 軌跡中最接近的位置。"""
    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法插入通訊點。")
        return gpx

    segment = gpx.tracks[0].segments[0]
    points = segment.points

    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        # 尋找最佳插入點
        for i in range(len(points) - 1):
            p1 = (points[i].latitude, points[i].longitude)
            p2 = (points[i + 1].latitude, points[i + 1].longitude)
            comm_p = (lat, lon)

            # 計算點到線段的距離（簡化為點到兩端點距離和）
            dist_sum = haversine(p1, comm_p, unit=Unit.METERS) + haversine(
                p2, comm_p, unit=Unit.METERS
            )

            if dist_sum < closest_dist:
                closest_dist = dist_sum
                insert_idx = i + 1

        if insert_idx is not None:
            new_point = gpxpy.gpx.GPXTrackPoint(
                latitude=lat, longitude=lon, elevation=ele
            )
            segment.points.insert(insert_idx, new_point)
            # print(f"    - 已插入通訊點：{label}")
    print(f"    - 已將所有通訊點插入軌跡")
    return gpx


# Step 3: 切割通訊點之間路線
def split_single_segment(
    gpx: gpxpy.gpx.GPX,
    start_latlon: Tuple[float, float],
    end_latlon: Tuple[float, float],
) -> gpxpy.gpx.GPX:
    """根據起點和終點的經緯度切割 GPX 軌跡。"""
    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_gpx.tracks.append(new_track)
    new_track.segments.append(new_segment)

    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法進行切割。")
        return new_gpx

    points = gpx.tracks[0].segments[0].points

    def find_idx(target: Tuple[float, float]) -> int:
        return min(
            range(len(points)),
            key=lambda i: haversine(
                (points[i].latitude, points[i].longitude), target, unit=Unit.METERS
            ),
        )

    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)

    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start

    new_segment.points = points[idx_start : idx_end + 1]
    print(f"    - 路線切割完成：從索引 {idx_start} 至 {idx_end}")
    return new_gpx


# Step 4: 匯出 GPX 與 CSV
def export_gpx_and_csv(gpx: gpxpy.gpx.GPX, gpx_name: str, csv_name: str):
    """將處理後的 GPX 物件儲存為 GPX 和 CSV 檔案。"""
    # 儲存 GPX
    with open(gpx_name, "w", encoding="utf-8") as f:
        f.write(gpx.to_xml())
    print(f"    - ✅ 儲存 GPX：{gpx_name}")

    # 匯出 CSV
    data = []
    if gpx.tracks and gpx.tracks[0].segments:
        for pt in gpx.tracks[0].segments[0].points:
            data.append(
                {
                    "latitude": pt.latitude,
                    "longitude": pt.longitude,
                    "elevation": pt.elevation,
                }
            )
    df = pd.DataFrame(data)
    df.to_csv(csv_name, index=False)
    print(f"    - ✅ 匯出 CSV：{csv_name}")


# Step 5: 顯示地圖（使用內建 LayerControl）
def visualize_with_checkbox(original_gpx, sliced_gpx, comm_pts, output_html):
    """產生包含原始路徑、切割路徑和通訊點的互動式地圖。"""
    # 找到地圖中心點
    if (
        original_gpx.tracks
        and original_gpx.tracks[0].segments
        and original_gpx.tracks[0].segments[0].points
    ):
        first_point = original_gpx.tracks[0].segments[0].points[0]
        map_location = [first_point.latitude, first_point.longitude]
    else:
        map_location = [23.5, 121]  # 如果沒有點，預設台灣中心

    m = folium.Map(location=map_location, zoom_start=14)

    # --- 建立圖層 ---
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # --- 將資料加入對應圖層 ---
    # 原始路線
    if original_gpx.tracks and original_gpx.tracks[0].segments:
        original_coords = [
            (pt.latitude, pt.longitude)
            for pt in original_gpx.tracks[0].segments[0].points
        ]
        folium.PolyLine(original_coords, color="blue", weight=4, opacity=0.6).add_to(
            original_layer
        )

    # 切割後路線
    if sliced_gpx.tracks and sliced_gpx.tracks[0].segments:
        cut_coords = [
            (pt.latitude, pt.longitude)
            for pt in sliced_gpx.tracks[0].segments[0].points
        ]
        folium.PolyLine(cut_coords, color="red", weight=4, opacity=0.8).add_to(
            cut_layer
        )

    # 通訊點 (使用 MarkerCluster)
    cluster = MarkerCluster().add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon],
            popup=f"{label}<br>海拔：約{ele} m",
            icon=folium.Icon(color="purple"),
        ).add_to(cluster)

    # --- 將圖層加入地圖 ---
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # --- 加入圖層控制器 (這會自動產生 Checkbox) ---
    folium.LayerControl().add_to(m)

    m.save(output_html)
    print(f"    - ✅ 地圖已儲存：{output_html}")


if __name__ == "__main__":
    # 🔹設定資料夾路徑
    gpx_folder = Path("hikingbook_route")
    txt_folder = Path("txt_data")
    gpx_output_folder = Path("gpx_data")
    csv_output_folder = Path("csv_data")
    html_output_folder = Path("Html_data")

    # 🔹建立輸出資料夾（若不存在）
    gpx_output_folder.mkdir(exist_ok=True)
    csv_output_folder.mkdir(exist_ok=True)
    html_output_folder.mkdir(exist_ok=True)

    print("--- 開始批次處理 ---")
    print(f"🔍 目前工作目錄: {os.getcwd()}")
    print(f"📂 正在從 '{gpx_folder.resolve()}' 讀取 GPX 檔案")
    print(f"📂 正在從 '{txt_folder.resolve()}' 讀取 TXT 檔案")
    print("-" * 40)

    # 🔹 尋找所有 .gpx 檔案
    gpx_files = list(gpx_folder.glob("*.gpx"))

    if not gpx_files:
        print("❌ 錯誤：在 'hikingbook_route' 資料夾中找不到任何 .gpx 檔案。")
        print("   請確認：")
        print("   1. 'hikingbook_route' 資料夾與您的 Python 腳本位於同一層級。")
        print("   2. 資料夾內確實有 .gpx 結尾的檔案。")
    else:
        print(f"✅ 找到 {len(gpx_files)} 個 GPX 檔案，準備開始處理...\n")

    # 🔹迴圈處理每個 GPX 檔案
    for gpx_file in gpx_files:
        base_name = gpx_file.stem  # 檔名不含副檔名
        txt_file = txt_folder / f"{base_name}.txt"

        print(f"🔸 正在處理: {gpx_file.name}")
        print(f"   - 正在尋找對應的 TXT: {txt_file.name}")

        if not txt_file.exists():
            print(f"   - ⚠️ 警告: 找不到對應的 TXT 檔案，略過此 GPX 檔案。")
            print("-" * 40)
            continue

        print(f"   - ✓ 找到對應的 TXT 檔案。")

        # 1. 去除時間
        gpx_obj = remove_time(str(gpx_file))

        # 2. 插入通訊點
        comm_pts = load_communication_points(str(txt_file))
        if not comm_pts:
            print(f"   - ⚠️ 警告: 未能從 {txt_file.name} 讀取任何通訊點，略過此檔案。")
            print("-" * 40)
            continue

        gpx_with_pts = insert_points(gpx_obj, comm_pts)

        # 3. 切出路段
        start_latlon = (comm_pts[0][1], comm_pts[0][2])
        end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
        sliced_gpx = split_single_segment(gpx_with_pts, start_latlon, end_latlon)

        # 4. 匯出 GPX + CSV
        gpx_outfile = gpx_output_folder / f"{base_name}_processed.gpx"
        csv_outfile = csv_output_folder / f"{base_name}_processed.csv"
        export_gpx_and_csv(sliced_gpx, str(gpx_outfile), str(csv_outfile))

        # 5. 儲存地圖 HTML
        html_outfile = html_output_folder / f"{base_name}_map.html"
        visualize_with_checkbox(gpx_with_pts, sliced_gpx, comm_pts, str(html_outfile))

        print("-" * 40)

    print("\n🏁 批次處理完成")

--- 開始批次處理 ---
🔍 目前工作目錄: c:\資展\專題\特徵
📂 正在從 'C:\資展\專題\特徵\hikingbook_route' 讀取 GPX 檔案
📂 正在從 'C:\資展\專題\特徵\txt_data' 讀取 TXT 檔案
----------------------------------------
✅ 找到 5 個 GPX 檔案，準備開始處理...

🔸 正在處理: chiyou_pintian.gpx
   - 正在尋找對應的 TXT: chiyou_pintian.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 7 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 6 至 83
    - ✅ 儲存 GPX：gpx_data\chiyou_pintian_processed.gpx
    - ✅ 匯出 CSV：csv_data\chiyou_pintian_processed.csv
    - ✅ 地圖已儲存：Html_data\chiyou_pintian_map.html
----------------------------------------
🔸 正在處理: tao.gpx
   - 正在尋找對應的 TXT: tao.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 4 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 7 至 165
    - ✅ 儲存 GPX：gpx_data\tao_processed.gpx
    - ✅ 匯出 CSV：csv_data\tao_processed.csv
    - ✅ 地圖已儲存：Html_data\tao_map.html
----------------------------------------
🔸 正在處理: tao_kalaye.gpx
   - 正在尋找對應的 TXT: tao_kalaye.txt
   - ✓ 找到對應的 TXT 檔案。
    - 已移除時間戳記
    - 成功讀取 8 個通訊點
    - 已將所有通訊點插入軌跡
    - 路線切割完成：從索引 7 至 182
   

gpt


In [5]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import gpxpy
import gpxpy.gpx
import folium
import numpy as np
from scipy.interpolate import interp1d

# --- 1. 設定資料夾 ---
GPX_FOLDER = Path("hikingbook_route")
TXT_FOLDER = Path("txt_data")
GPX_OUT = Path("gpx_data")
CSV_OUT = Path("csv_data")
HTML_OUT = Path("Html_data")
TXT_FIN = Path("TXT_fin")

for folder in [GPX_OUT, CSV_OUT, HTML_OUT, TXT_FIN]:
    folder.mkdir(parents=True, exist_ok=True)


# --- 2. 讀取 GPX 轉 GeoDataFrame ---
def gpx_to_gdf(gpx_path: Path) -> gpd.GeoDataFrame:
    with open(gpx_path, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    rows = []
    for track in gpx.tracks:
        for segment in track.segments:
            for idx, pt in enumerate(segment.points):
                rows.append(
                    {
                        "latitude": pt.latitude,
                        "longitude": pt.longitude,
                        "elevation": pt.elevation,
                        "geometry": Point(pt.longitude, pt.latitude),
                        "point_index": idx,
                    }
                )
    gdf = gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")
    return gdf


# --- 3. 讀取 TXT 通訊點(不含海拔)轉 GeoDataFrame ---
def txt_to_gdf(txt_path: Path) -> gpd.GeoDataFrame:
    df = pd.read_csv(txt_path, sep="\t", encoding="utf-8")
    # 強制轉成字串，避免欄位問題
    df["緯度"] = pd.to_numeric(df["緯度"], errors="coerce")
    df["經度"] = pd.to_numeric(df["經度"], errors="coerce")
    # 不讀海拔，丟棄該欄位或不使用
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df["經度"], df["緯度"]), crs="EPSG:4326"
    )
    return gdf


# --- 4. 路徑修剪 + 插入通訊點 ---
def trim_and_insert(
    route_gdf: gpd.GeoDataFrame, comm_gdf: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    # 先轉成 LineString
    line = LineString(route_gdf.geometry.tolist())

    # 找通訊點中最後一個點投影在線上的距離作為修剪終點
    last_comm_pt = comm_gdf.geometry.iloc[-1]
    trim_dist = line.project(last_comm_pt)

    # 修剪 LineString：從0到trim_dist的部分
    trimmed_line = line.interpolate(0), line.interpolate(trim_dist)
    trimmed_coords = [pt for pt in line.coords if line.project(Point(pt)) <= trim_dist]
    # 為避免沒剛好有終點，手動加最後點
    trimmed_coords.append((last_comm_pt.x, last_comm_pt.y))

    trimmed_line = LineString(trimmed_coords)

    # 將 trimmed_line 轉為 GeoDataFrame
    trimmed_gdf = gpd.GeoDataFrame(
        {"geometry": list(map(Point, trimmed_line.coords))}, crs="EPSG:4326"
    )

    # 插入通訊點：對每個通訊點找最接近的線段位置，插入排序
    all_points = list(trimmed_gdf.geometry) + list(comm_gdf.geometry)
    # 去重並排序
    unique_points = list(set(all_points))
    unique_points.sort(key=lambda pt: trimmed_line.project(pt))

    gdf_final = gpd.GeoDataFrame({"geometry": unique_points}, crs="EPSG:4326")
    return gdf_final


# --- 5. 高程內插 ---
def interpolate_elevation(
    route_gdf: gpd.GeoDataFrame, base_gdf: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    # base_gdf有高程，route_gdf點可能缺高程

    # 建立基準距離與高程對應（以base_gdf為主）
    line = LineString(base_gdf.geometry.tolist())
    base_gdf["dist"] = base_gdf.geometry.apply(lambda p: line.project(p))
    base_gdf = base_gdf.sort_values("dist")

    f_interp = interp1d(
        base_gdf["dist"], base_gdf["elevation"], kind="linear", fill_value="extrapolate"
    )

    route_gdf["dist"] = route_gdf.geometry.apply(lambda p: line.project(p))
    route_gdf["elevation"] = route_gdf["dist"].apply(f_interp)

    route_gdf.drop(columns=["dist"], inplace=True)
    return route_gdf


# --- 6. 匯出 GPX ---
def gdf_to_gpx(gdf: gpd.GeoDataFrame, out_path: Path):
    gpx = gpxpy.gpx.GPX()
    track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(track)
    segment = gpxpy.gpx.GPXTrackSegment()
    track.segments.append(segment)
    for _, row in gdf.iterrows():
        pt = gpxpy.gpx.GPXTrackPoint(
            latitude=row.geometry.y,
            longitude=row.geometry.x,
            elevation=float(row.elevation) if "elevation" in row else None,
        )
        segment.points.append(pt)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(gpx.to_xml())


# --- 7. 匯出 CSV ---
def gdf_to_csv(gdf: gpd.GeoDataFrame, out_path: Path):
    df = gdf.copy()
    df["latitude"] = df.geometry.y
    df["longitude"] = df.geometry.x
    df.to_csv(out_path, index=False, encoding="utf-8-sig")


# --- 8. 產生互動地圖 ---
def create_map(original_gdf, final_gdf, comm_gdf, out_path):
    m = folium.Map(
        location=[original_gdf.geometry.y.iloc[0], original_gdf.geometry.x.iloc[0]],
        zoom_start=14,
    )

    fg_original = folium.FeatureGroup(name="原始路徑 (藍色)", show=True)
    folium.PolyLine(
        [(pt.y, pt.x) for pt in original_gdf.geometry], color="blue"
    ).add_to(fg_original)
    fg_original.add_to(m)

    fg_final = folium.FeatureGroup(name="處理後路徑 (紅色)", show=True)
    folium.PolyLine([(pt.y, pt.x) for pt in final_gdf.geometry], color="red").add_to(
        fg_final
    )
    fg_final.add_to(m)

    fg_comm = folium.FeatureGroup(name="通訊點 (紫色)", show=True)
    for _, row in comm_gdf.iterrows():
        folium.CircleMarker(
            location=(row.geometry.y, row.geometry.x),
            radius=6,
            color="purple",
            fill=True,
            fill_opacity=0.8,
            tooltip=row.get("路標指示", "通訊點"),
        ).add_to(fg_comm)
    fg_comm.add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)

    m.save(out_path)


# --- 9. 輸出含高程 TXT ---
def gdf_to_txt(gdf: gpd.GeoDataFrame, out_path: Path):
    df = gdf.copy()
    df["緯度"] = df.geometry.y
    df["經度"] = df.geometry.x
    # 假設你想輸出這幾欄，且高程用 elevation
    export_cols = ["緯度", "經度"]
    if "elevation" in df.columns:
        df["海拔"] = df["elevation"]
        export_cols.append("海拔")
    df.to_csv(
        out_path, sep="\t", index=False, encoding="utf-8-sig", columns=export_cols
    )


# --- 10. 主程式 ---
def main():
    for gpx_file in GPX_FOLDER.glob("*.gpx"):
        txt_file = TXT_FOLDER / (gpx_file.stem + ".txt")
        if not txt_file.exists():
            print(f"跳過 {gpx_file.name}，找不到對應 TXT 檔")
            continue

        print(f"處理 {gpx_file.name} 與 {txt_file.name}")

        # 讀取 GPX -> GeoDataFrame
        gdf_route = gpx_to_gdf(gpx_file)

        # 讀取 TXT -> GeoDataFrame (不含海拔)
        gdf_comm = txt_to_gdf(txt_file)

        # 路徑修剪與插入
        gdf_final = trim_and_insert(gdf_route, gdf_comm)

        # 高程內插
        gdf_final = interpolate_elevation(gdf_final, gdf_route)

        # 輸出 GPX
        gpx_out_path = GPX_OUT / f"{gpx_file.stem}_final.gpx"
        gdf_to_gpx(gdf_final, gpx_out_path)

        # 輸出 CSV
        csv_out_path = CSV_OUT / f"{gpx_file.stem}_final.csv"
        gdf_to_csv(gdf_final, csv_out_path)

        # 產生互動地圖
        html_out_path = HTML_OUT / f"{gpx_file.stem}_map.html"
        create_map(gdf_route, gdf_final, gdf_comm, html_out_path)

        # 輸出含高程 TXT
        txt_out_path = TXT_FIN / f"{gpx_file.stem}_final.txt"
        gdf_to_txt(gdf_final, txt_out_path)

        print(f"完成：{gpx_file.name}")


if __name__ == "__main__":
    main()

處理 chiyou_pintian.gpx 與 chiyou_pintian.txt
完成：chiyou_pintian.gpx
處理 tao.gpx 與 tao.txt
完成：tao.gpx
處理 tao_kalaye.gpx 與 tao_kalaye.txt
完成：tao_kalaye.gpx
處理 tao_waterfall.gpx 與 tao_waterfall.txt
完成：tao_waterfall.gpx
跳過 品田池有.gpx，找不到對應 TXT 檔


c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
